## Conservation Genomics Practical

------------------------ 
**Background**:

The primary data for modern conservation genomics efforts often includes whole-genome
sequences of multiple individuals of a threatened species. From this data, we infer the levels
and patterns of genetic diversity, and apply population genetics models to infer past
demography. We can also infer levels of inbreeding and patterns of migration from sequence
data. If we have access to information about the deleterious consequences of variants, or we
apply computation inference of those deleterious effects, we can further infer
<br>
<br>
I have four sets of sequence data, all drawn from one chromosome, from four different populations with distinct population histories. The data consist of only the segregating sites, with “0” being the ancestral allele, and “1” being the derived allele. Each row represents a haplotype drawn from the population, and each dataset consists of 50 such
haplotypes.
I will randomly select one of the blocks from each data file, and, for each data set, I will:
1. Read in the data into a matrix
2. Calculate the nucleotide diversity, pi (the data are the segregating sites **only**, out of a span of 10 kb).
3. Determine the number of segregating sites, S.
4. Calculate Watterson’s theta.
5. Calculate Tajima’s D

**Equations:** <br>
Watterson's estimator ($\theta$) <br>
&nbsp; $\theta=\frac{S}{\sum \limits_{i=1}^{n-1}1/i}$ &nbsp; ; &nbsp; where $S$=segregating sites, $n$=the number of loci (columns). <br>
<br>
Average nucleotide diversity ($\pi$) <br>
&nbsp; $ \pi=\frac{1}{L} \sum\limits_{k=1}^{L}h_k$ &nbsp; ; &nbsp; where $h_{k}=\frac{2p(n-p)}{n(n-1)}$ , $L$=number of loci (i.e. nucelotide positions / columns), $n$=sample size (number of haplotypes/rows) , and $p$=number of derived alleles (1's) at loci $k$. <br>
<br>
Tajima's $D$ <br>
$D=\frac{\pi-\theta}{SD(\pi-\theta)}$ 

In [3]:
#imports
import numpy as np
import pandas as pd
from itertools import combinations
import random
#from scipy.stats import tajima_d

In [1]:
# Input matrices 
    # from each file, select one block at random, 
    # each block consists of haplotypes from 50 individuals (rows)

# Need to do the following for all 4 blocks selected from the 4 files.
hapmx = pd.read_csv('ConGensData1block1.txt', sep = ',', header=None)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Calculate nucelotide diversity (pi)
def calc_pi(haplotypes):
    # variables
    # formula
    # calculations
    return pi

result_pi = calc_pi(hapmx)
print(f"Nucelotide diversity (pi): {result_pi}")

In [ ]:
# Calculate the number of segregating sites (S)
def count_segregating_sites(haplotypes):
    n = len(haplotypes.axes[1])
    S = 0
    for i in range(0,n):
        if len(haplotypes[i].unique()) == 1: 
            S = S
        else:
            S = S + 1
    return S

result_S = count_segregating_sites(hapmx)
print(f"Segregating sites: {result_S} out of" {n} "sites.")

In [ ]:
# Calculate wattersons theta
def calc_watterson(haplotypes):
    # S = segregating sites
    # a = sum(1/i)
    # pi = S / a ...
    return watt

result_watt = calculate_watterson(hapmx)
print(f"Wattersons theta: {result_watt}")

In [ ]:
# Calculate Tajima's D
def tajimas_d(haplotypes):
    # variable
    # formula
    # calculations
    Tajimas_D = numerator / denominator
    return Tajimas_D

result_D = tajimas_d(haplotype_matrix)
print(f"Tajima's D: {result_D}")